In [ ]:
%pip install openai

In [1]:
import pandas as pd
import json
import numpy as np
from time import sleep
import time
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=open("API_key", "r").read(),
)

In [2]:
# Load the test datasets
test_en = pd.read_json("../../datasets/copa-en-test.jsonl", lines=True)
test_sl = pd.read_json("../../datasets/copa-sl-test.jsonl", lines=True)
test_hr_ckm = pd.read_json("../../datasets/copa-hr-ckm-test.jsonl", lines=True)
test_hr = pd.read_json("../../datasets/copa-hr-test.jsonl", lines=True)
test_mk = pd.read_json("../../datasets/copa-mk-test.jsonl", lines=True)
test_sl_cer = pd.read_json("../../datasets/copa-sl-cer-test.jsonl", lines=True)
test_sr = pd.read_json("../../datasets/copa-sr-test.jsonl", lines=True)
test_sr_tor = pd.read_json("../../datasets/copa-sr-tor-test.jsonl", lines=True)

print(test_en.shape, test_sl.shape, test_hr.shape, test_hr_ckm.shape, test_mk.shape, test_sl_cer.shape, test_sr.shape, test_sr_tor.shape)

(500, 5) (500, 5) (500, 7) (500, 5) (500, 7) (500, 5) (500, 7) (500, 5)


In [3]:
def predict_gpt(df_test_name, gpt_model):

	df_path = f"../../datasets/{df_test_name}-test.jsonl"

	responses = []

	start_time = time.time()
	for line in open(df_path):
		entry=json.loads(line)

		if df_test_name != "copa-en":
			prompt= 'You will be given a task. The task definition is in English, but the task itself is in another language. Here is the task!\nGiven the premise "'+entry['premise']+'",'
			if entry['question']=='cause':
				prompt+=' and that we are looking for the cause of this premise,'
			else:
				prompt+=' and that we are looking for the result of this premise, '
			prompt+=f"""which hypothesis is more plausible?\nHypothesis 1: "{entry['choice1']}".\nHypothesis 2: "{entry['choice2']}".
					
			### Output format
				Return a valid JSON dictionary with the following key: 'answer' and a value should be an integer -- either 1 (if hypothesis 1 is more plausible) or 2 (if hypothesis 2 is more plausible).
			"""
		elif df_test_name == "copa-en":
			prompt= 'You will be given a task. The task definition is in English, as is the task itself. Here is the task!\nGiven the premise "'+entry['premise']+'",'
			if entry['question']=='cause':
				prompt+=' and that we are looking for the cause of this premise,'
			else:
				prompt+=' and that we are looking for the result of this premise,'
			prompt+=f"""which hypothesis is more plausible?\nHypothesis 1: "{entry['choice1']}".\nHypothesis 2: "{entry['choice2']}".
					
			### Output format
				Return a valid JSON dictionary with the following key: 'answer' and a value should be an integer -- either 1 (if hypothesis 1 is more plausible) or 2 (if hypothesis 2 is more plausible).
			"""
		
		if gpt_model != "anthropic/claude-haiku-4.5":
			completion = client.chat.completions.create(model=gpt_model,
				response_format= {"type": "json_object"},
				messages=[
				{
					"role": "user",
					"content": prompt}
				],
				temperature = 0
			)
		# anthropic requires additional rules, as it doesn't support json output formatting
		elif gpt_model == "anthropic/claude-haiku-4.5":
			prompt+= "Answer ONLY with the JSON dictionary, do NOT provide your reasoning or the explanation. Provide only the dictionary with the integer 1 or 2 as the answer!"
			completion = client.chat.completions.create(model=gpt_model,
				response_format= {"type": "json_object"},
				stop="}",
				messages=[
				{
					"role": "user",
					"content": prompt}
				],
				temperature = 0)

		initial_response=completion.choices[0].message.content

		response = initial_response.replace("\n", "")
		response = response.replace("\t", "")

		# Get out a label
		try:
			# Convert the string into a dictionary
			response = json.loads(response)
			
			# The true labels are 0 or 1, so you have to change the answer by substracting 1 from it.
			predicted = response["answer"]-1
			responses.append(predicted)
		# add a possibility of something going wrong
		except:
			predicted = initial_response
			print("error with extracting a label:")
			print(initial_response)
			responses.append(initial_response)

	end_time = time.time()
	elapsed_time_min = end_time-start_time

	print(f"Prediction finished. It took {elapsed_time_min/60} min for 500 instances - {elapsed_time_min/500} s per instance.")

	# Create a json with results

	current_results = {
		"system": gpt_model,
		"predictions": [
			{
			"train": "NA (zero-shot)",
			"test": "{}".format(df_test_name),
			"predictions": responses,
			}
		]
		}

	# The only thing that needs to be changed in the code from OpenAI
	gpt_model_name = gpt_model.split("/")[1]

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format(gpt_model_name, df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format(gpt_model_name, df_test_name))

In [5]:
tests = ["copa-en","copa-sl", "copa-hr", "copa-hr-ckm", "copa-mk", "copa-sl-cer", "copa-sr", "copa-sr-tor"]

In [6]:
# First, let's evaluate the models that we evaluated in other tasks as well
models = ["google/gemini-2.5-flash", "mistralai/mistral-medium-3.1"]

In [7]:
# Some were already done, continue with others
for test in tests[4:]:
	print(test)
	for model in models:
		print(model)
		predict_gpt(test, model)

copa-mk
google/gemini-2.5-flash
Prediction finished. It took 4.317918360233307 min for 500 instances - 0.5181502032279969 s per instance.
Classification with gemini-2.5-flash on copa-mk finished.
mistralai/mistral-medium-3.1
Prediction finished. It took 6.425474607944489 min for 500 instances - 0.7710569529533386 s per instance.
Classification with mistral-medium-3.1 on copa-mk finished.
copa-sl-cer
google/gemini-2.5-flash
Prediction finished. It took 4.583541476726532 min for 500 instances - 0.5500249772071838 s per instance.
Classification with gemini-2.5-flash on copa-sl-cer finished.
mistralai/mistral-medium-3.1
Prediction finished. It took 6.624265456199646 min for 500 instances - 0.7949118547439575 s per instance.
Classification with mistral-medium-3.1 on copa-sl-cer finished.
copa-sr
google/gemini-2.5-flash
Prediction finished. It took 4.576912299791972 min for 500 instances - 0.5492294759750366 s per instance.
Classification with gemini-2.5-flash on copa-sr finished.
mistralai/

In [ ]:
# Evaluate also the Anthropic model

for test in tests[1:]:
	print(test)
	predict_gpt(test, "anthropic/claude-haiku-4.5")

In [6]:
# Add the PRO Gemini model

for test in tests:
	print(test)
	predict_gpt(test, "google/gemini-2.5-pro")

copa-en
error with extracting a label:

error with extracting a label:

Prediction finished. It took 83.85881508191427 min for 500 instances - 10.063057809829711 s per instance.
Classification with gemini-2.5-pro on copa-en finished.
copa-sl
error with extracting a label:



APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient credits. Add more using https://openrouter.ai/settings/credits', 'code': 402}}